## Projet 5 : Analyser les ecarts de performance entre departements 

### Problematique metier : 

UNe entreprise souhaite analyser les ecarts de performance entre ses departements
en explorant les relations entre variables qualitatives(départements, satisfaction)
et quantitatives (salaire, temps de travail).

### objectif : 
valider les differences en appliquant des tests statistiques

In [2]:
 # importation des librairies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [3]:
# chargement des données

df = pd.read_csv("Extended_Employee_Performance_and_Productivity_Data.csv")

df.head()

,Employee_ID,Department,Gender,Age,Job_Title,Hire_Date,Years_At_Company,Education_Level,Performance_Score,Monthly_Salary,Work_Hours_Per_Week,Projects_Handled,Overtime_Hours,Sick_Days,Remote_Work_Frequency,Team_Size,Training_Hours,Promotions,Employee_Satisfaction_Score,Resigned
0,1,IT,Male,55,Specialist,2022-01-19 08:03:05.556036,2,High School,5,6750.0,33,32,22,2,0,14,66,0,2.63,False
1,2,Finance,Male,29,Developer,2024-04-18 08:03:05.556036,0,High School,5,7500.0,34,34,13,14,100,12,61,2,1.72,False
2,3,Finance,Male,55,Specialist,2015-10-26 08:03:05.556036,8,High School,3,5850.0,37,27,6,3,50,10,1,0,3.17,False
3,4,Customer Support,Female,48,Analyst,2016-10-22 08:03:05.556036,7,Bachelor,2,4800.0,52,10,28,12,100,10,0,1,1.86,False
4,5,Engineering,Female,36,Analyst,2021-07-23 08:03:05.556036,3,Bachelor,2,4800.0,38,11,29,13,100,15,9,1,1.25,False


## Explorations des donnees 

In [4]:
# affichage des statistiques descriptives
print(f"Shape: {df.shape}")
print("\nInformations:")
print(df.info())
print("\nStatistiques descriptives:")
print(df.describe(include='all'))

Shape: (100000, 20)

Informations:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Employee_ID                  100000 non-null  int64  
 1   Department                   100000 non-null  object 
 2   Gender                       100000 non-null  object 
 3   Age                          100000 non-null  int64  
 4   Job_Title                    100000 non-null  object 
 5   Hire_Date                    100000 non-null  object 
 6   Years_At_Company             100000 non-null  int64  
 7   Education_Level              100000 non-null  object 
 8   Performance_Score            100000 non-null  int64  
 9   Monthly_Salary               100000 non-null  float64
 10  Work_Hours_Per_Week          100000 non-null  int64  
 11  Projects_Handled             100000 non-null  int64  
 12  Overtime_Hours          

In [5]:
# afficher tout les noms de colonnes 
print(df.columns)

Index(['Employee_ID', 'Department', 'Gender', 'Age', 'Job_Title', 'Hire_Date',
       'Years_At_Company', 'Education_Level', 'Performance_Score',
       'Monthly_Salary', 'Work_Hours_Per_Week', 'Projects_Handled',
       'Overtime_Hours', 'Sick_Days', 'Remote_Work_Frequency', 'Team_Size',
       'Training_Hours', 'Promotions', 'Employee_Satisfaction_Score',
       'Resigned'],
      dtype='object')


In [6]:
# afficher les colonnes : Departement , employee_satisfaction_score, performance scores 

print(df[["Department", "Employee_Satisfaction_Score", "Performance_Score"]])

             Department  Employee_Satisfaction_Score  Performance_Score
0                    IT                         2.63                  5
1               Finance                         1.72                  5
2               Finance                         3.17                  3
3      Customer Support                         1.86                  2
4           Engineering                         1.25                  2
...                 ...                          ...                ...
99995           Finance                         1.28                  4
99996                IT                         3.48                  5
99997        Operations                         2.60                  2
99998                HR                         3.10                  5
99999           Finance                         2.64                  1

[100000 rows x 3 columns]


### Analyse des relations entre les variables qualitatives 

### A - test de khi2

en effectuant le test de khi2, on essayeras de voir si il y a une dependence entre la satisfaction des employes et leurs departements.

In [7]:
# test de khi2 sur  la satisfactions des employes et leurs departements
from scipy.stats import chi2_contingency

# calcul de la matrice de contingence
contingency_matrix = pd.crosstab(df['Department'], df['Employee_Satisfaction_Score'])

# test de khi2
chi2, p, dof, expected = chi2_contingency(contingency_matrix)

# affichage des resultats
print(f"Chi2: {chi2:.2f}")
print(f"p-value: {p:.2f}")


Chi2: 3204.30
p-value: 0.48


Selon le test de khi2 observer, il n'y a pas de difference significative entre les satisfactions des employes des differents departements.La statistique chi2 3204.30 montre l'ampleur de la difference entre les frequences observées et les frequences attendues. plus la valeur est grandes, 
plus la difference l'est 

etant donne que la p-value est superieur a 0.05, on peut conclure que la difference n'est pas significative

### B - V de cramer 

afin d'observer si quel pourrais etre la force assosiative entre les variables Department et Employee_Satisfaction_Score, on utilise la fonction v de cramer

In [8]:
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k - 1, r - 1))

# Calcul de la matrice de contingence
contingency_matrix = pd.crosstab(df['Department'], df['Employee_Satisfaction_Score'])

# Calcul du coefficient de Cramér
cramer_v = cramers_v(contingency_matrix)

print(f"Coefficient de Cramer: {cramer_v:.2f}")

Coefficient de Cramer: 0.06


Le coefficinet de cramer de 0.06 nous confirme donc que la relation entre ces deux variables est faible.

le coefficient de cramer varie de 0 a 1 ou 0 indique aucune association et 1 une association parfaite 

###  Comparaison des moyennes ( quali - quanti)

#### A - analyse de variance  ANOVA

l'analyse de variance ANOVA est une methode statistique qui compare les moyennes entre
plus de deux groupes (données normalement distribuées et
variances homogènes)

ce que l'on feras donc est : 

1-Normalité : On teste la distribution des scores pour chaque département avec le test de Shapiro-Wilk.

2-Homogénéité des variances : On vérifie que les variances sont similaires entre les groupes avec le test de Levene.

3-ANOVA : On réalise le test ANOVA pour comparer les moyennes.

4-Post-hoc : En cas de résultat significatif, on applique le test de Tukey pour identifier précisément les différences.


In [12]:
print(df.columns)

Index(['Employee_ID', 'Department', 'Gender', 'Age', 'Job_Title', 'Hire_Date',
       'Years_At_Company', 'Education_Level', 'Performance_Score',
       'Monthly_Salary', 'Work_Hours_Per_Week', 'Projects_Handled',
       'Overtime_Hours', 'Sick_Days', 'Remote_Work_Frequency', 'Team_Size',
       'Training_Hours', 'Promotions', 'Employee_Satisfaction_Score',
       'Resigned'],
      dtype='object')


In [13]:
# ANOVA 

from scipy.stats import f_oneway 

# les colonnes interessantes 

data = df[['Department', 'Performance_Score']]

# liste des Departments pour iterer dessus 

Departments = data['Department'].unique()

# verification de la normalite des scores pour chaque Department ( test de shapiro-wilk)
print("=== Test de normaliter par Departments ===")
for dept in Departments: 
    dept_data = data[data['Department'] == dept]['Performance_Score']
    stat, p_value = stats.shapiro(dept_data)
    print(f"Department {dept} - Statistique = {stat:.4f}, p-value={p_value:.4f}")
    # remarque sur l'interpretation 
    if p-value >  0.05: 
        print(" -> Distribution conforme a la normalite")
    else: 
        print(" -> Distribution non conforme a la normalite")

    print("-"*50)

# Verification de l'homogeneiter des variances (Test de Levene)

print("\n=== Test d'homogeneite des variances ===")
groups = [data[data['Department'] == dept ]['Performance_Scrore']for dept in Departments]
Levene_stat,levene_p = stats.levene(*groups)
print(f"Test de levene - Statistique={levene_stat:.4f}, p-value={levene_p:.4f}")
if levene_p > 0.05:
    print("Les variances sont Homogene")
else: 
    print("Attention : Les variances different significativements entre les groupes.")

# Realisation de l'ANOVA 
print("\n=== Realisation de l'analyse de variance ===")
model = ols('Performance_Scores ~ C(Department)', data=data).fit()
anova_table = sm.stats.anova_lm(model, type=2)
print(anova_table)

# Analyse post-hoc : si ANOVA est significative donc in ferieur au seuil de 5% 
anova_p_value = anova_table["PR(>f)"][0]
if anova_p_value < 0.05: 
    print("\nL'ANOVA est significative (p < 0.05). passage au test post-hoc pour identifier les differences entre Departments ")
    tukey  = pairwise_tukeyhsd(endog=data['Performance_Score'], groups=data['Departments'],alpha=0.05)
    print(tukey)
    # option1 : affichage graphique du resultat du test de Tukey (post-hoc)
    tukey.plot_simultaneous(figsize=(10, 6))
    plt.title("Tuket HSD - Comparaison des moyennes par Department")
    plt.xlabel("Difference de Score de Performance")
    plt.show()
else:
    print("\nL'ANOVA n'est pas significative (p >= 0.05). Aucun test post-hoc n'est necessaire. ")

=== Test de normaliter par Departments ===
Department IT - Statistique = 0.8861, p-value=0.0000


C:\Users\USER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 11131.
  res = hypotest_fun_out(*samples, **kwds)


NameError: name 'value' is not defined